In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

from pathlib import Path

from utils.data_loader import create_data_loaders, create_exp_loaders
from model.KnowNet import KnowNet
from setting.base_config import BaseConfig

Get DataLoader

In [2]:
config = BaseConfig()
args = config.get_config(2017)
data_loader , label_encoder, group = create_data_loaders(2017, args)
data_loader

[(<torch.utils.data.dataloader.DataLoader at 0x7f2fb0438c10>,
  <torch.utils.data.dataloader.DataLoader at 0x7f2fb0438a00>),
 (<torch.utils.data.dataloader.DataLoader at 0x7f2e4d309a30>,
  <torch.utils.data.dataloader.DataLoader at 0x7f2e4d309f10>),
 (<torch.utils.data.dataloader.DataLoader at 0x7f2e4873d730>,
  <torch.utils.data.dataloader.DataLoader at 0x7f2e4873d340>),
 (<torch.utils.data.dataloader.DataLoader at 0x7f2e4873d310>,
  <torch.utils.data.dataloader.DataLoader at 0x7f2e4873d250>),
 (<torch.utils.data.dataloader.DataLoader at 0x7f2e4873d3a0>,
  <torch.utils.data.dataloader.DataLoader at 0x7f2e4873d190>)]

In [3]:
GPU_NUM=1
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
layers = [77, 1000, 538]

net = KnowNet(group, layers)
net = net.to(device)
device

device(type='cuda', index=1)

In [4]:
def accuracy(x, y):
    return (torch.argmax(x, axis = 1) == y).float().sum()/x.size()[0]

In [8]:
train_data, val_data = data_loader[1]

train_loss = []
val_loss = []
train_acc = []
val_acc = []


criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr = 0.0001)
for epoch in range(30):
    for data, label in train_data:
        data, label = data.to(device), label.to(device)
        output = net(data)
        loss = criterion(output, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    acc = 0.
    for data, label in val_data:
        data, label = data.to(device), label.to(device)
        output = net(data)
        acc += accuracy(output, label).item()
    acc /= len(val_data)
    print("{} : val acc = {}".format(epoch, acc))

0 : val acc = 0.0431578947368421
1 : val acc = 0.06578947368421052
2 : val acc = 0.08157894736842106
3 : val acc = 0.09736842105263158
4 : val acc = 0.11421052631578947
5 : val acc = 0.12368421052631579
6 : val acc = 0.13526315789473684
7 : val acc = 0.13736842105263158
8 : val acc = 0.15947368421052632
9 : val acc = 0.18052631578947367
10 : val acc = 0.18052631578947367
11 : val acc = 0.18842105263157893
12 : val acc = 0.2
13 : val acc = 0.2
14 : val acc = 0.2031578947368421
15 : val acc = 0.21736842105263157
16 : val acc = 0.22052631578947368
17 : val acc = 0.21263157894736842
18 : val acc = 0.23473684210526316
19 : val acc = 0.2326315789473684
20 : val acc = 0.23210526315789473
21 : val acc = 0.24157894736842106
22 : val acc = 0.24421052631578946
23 : val acc = 0.25
24 : val acc = 0.2468421052631579
25 : val acc = 0.2578947368421053
26 : val acc = 0.26631578947368423
27 : val acc = 0.25263157894736843
28 : val acc = 0.26789473684210524
29 : val acc = 0.2710526315789474


In [5]:
from setting.exp_config import ExpConfig
from exp.tuner import tune_hyperparameter

In [6]:
config = ExpConfig()
args = config.get_config(2017)

net = lambda layer: KnowNet(group, layer)
hyper_params_setting = config.get_exp_arg(net)
default_setting = config.get_default_arg()

In [7]:
args

{'FOLD': 5,
 'DATA_ROOT': PosixPath('../KNOW_data'),
 'STRING_INDEX': [87, 88, 89, 92, 121, 139, 140, 141, 142, 143, 148],
 'batch_size': 4,
 'num_workers': 1,
 'device': device(type='cuda', index=1),
 'early_stopping_step': 5}

In [8]:
default_setting

{'batch_size': 4,
 'learning_rate': 0.0001,
 'weight_decay': 5e-06,
 'criterion': CrossEntropyLoss(),
 'optimizer': <function setting.exp_config.ExpConfig.__init__.<locals>.<lambda>(p, lr, wd)>,
 'model': KnowNet(
   (small_net_list): ModuleList(
     (0): Sequential(
       (0): Linear(in_features=2, out_features=4, bias=True)
       (1): ReLU()
       (2): Linear(in_features=4, out_features=1, bias=True)
       (3): ReLU()
     )
     (1): Sequential(
       (0): Linear(in_features=2, out_features=4, bias=True)
       (1): ReLU()
       (2): Linear(in_features=4, out_features=1, bias=True)
       (3): ReLU()
     )
     (2): Sequential(
       (0): Linear(in_features=2, out_features=4, bias=True)
       (1): ReLU()
       (2): Linear(in_features=4, out_features=1, bias=True)
       (3): ReLU()
     )
     (3): Sequential(
       (0): Linear(in_features=2, out_features=4, bias=True)
       (1): ReLU()
       (2): Linear(in_features=4, out_features=1, bias=True)
       (3): ReLU()
    

In [ ]:
train_loader = lambda batch_size: create_exp_loaders(2017, args, 1, True, batch_size)
val_loader = lambda batch_size: create_exp_loaders(2017, args, 1, False, batch_size)

tune_hyperparameter(
    train_dataset=train_loader, val_dataset=val_loader,  # data loader
    hyper_params_setting=hyper_params_setting, default_setting=default_setting,  # hyperparameters for tuning
    project='KNOW_dacon_contest', entity='ahagyue',  # names
    arguments=args
)

wandb: Currently logged in as: ahagyue (use `wandb login --relogin` to force relogin)


Create sweep with ID: n3h4t7kn
Sweep URL: https://wandb.ai/ahagyue/KNOW_dacon_contest/sweeps/n3h4t7kn


wandb: Agent Starting Run: or93tqlc with config:
wandb: 	batch_size: 4
wandb: 	criterion: 0
wandb: 	learning_rate: 0.0001
wandb: 	model: 0
wandb: 	optimizer: 0
wandb: 	weight_decay: 5e-06


Epoch: 0. Train.      Loss: 6.211 , time : 40.31659388542175: 100%|██████████| 1898/1898 [00:40<00:00, 47.04it/s] 
Epoch: 0. Validation. Loss: 6.256 , time : 2.15878963470459: 100%|██████████| 475/475 [00:02<00:00, 216.31it/s]  
Epoch: 1. Train.      Loss: 6.070 , time : 38.22487163543701: 100%|██████████| 1898/1898 [00:38<00:00, 49.61it/s] 
Epoch: 1. Validation. Loss: 6.254 , time : 2.131404399871826: 100%|██████████| 475/475 [00:02<00:00, 219.17it/s] 
Epoch: 2. Train.      Loss: 6.019 , time : 38.209322690963745: 100%|██████████| 1898/1898 [00:38<00:00, 49.63it/s]
Epoch: 2. Validation. Loss: 6.212 , time : 2.107211112976074: 100%|██████████| 475/475 [00:02<00:00, 221.69it/s] 
Epoch: 3. Train.      Loss: 5.954 , time : 38.271275758743286: 100%|██████████| 1898/1898 [00:38<00:00, 49.55it/s]
Epoch: 3. Validation. Loss: 6.146 , time : 2.1117544174194336: 100%|██████████| 475/475 [00:02<00:00, 221.21it/s]
Epoch: 4. Train.      Loss: 5.859 , time : 38.313408851623535: 100%|██████████| 1898

Epoch: 70. Validation. Loss: 4.048 , time : 2.1849021911621094: 100%|██████████| 475/475 [00:02<00:00, 213.91it/s]


...

wandb: Agent Starting Run: s1qx9o4v with config:
wandb: 	batch_size: 4
wandb: 	criterion: 0
wandb: 	learning_rate: 0.0001
wandb: 	model: 16
wandb: 	optimizer: 1
wandb: 	weight_decay: 5e-06


Epoch: 0. Train.      Loss: 0.969 , time : 23.822208642959595: 100%|██████████| 1898/1898 [00:23<00:00, 79.53it/s]
Epoch: 0. Validation. Loss: 3.975 , time : 2.2368693351745605: 100%|██████████| 475/475 [00:02<00:00, 208.16it/s]
Epoch: 1. Train.      Loss: 0.949 , time : 24.065962314605713: 100%|██████████| 1898/1898 [00:24<00:00, 78.72it/s]
Epoch: 1. Validation. Loss: 3.976 , time : 2.1809475421905518: 100%|██████████| 475/475 [00:02<00:00, 213.50it/s]
Epoch: 2. Train.      Loss: 0.938 , time : 23.751142501831055: 100%|██████████| 1898/1898 [00:23<00:00, 79.69it/s]
Epoch: 2. Validation. Loss: 3.957 , time : 2.204352855682373: 100%|██████████| 475/475 [00:02<00:00, 211.30it/s] 
Epoch: 3. Train.      Loss: 0.932 , time : 23.979156017303467: 100%|██████████| 1898/1898 [00:24<00:00, 79.01it/s]
Epoch: 3. Validation. Loss: 3.959 , time : 2.1478099822998047: 100%|██████████| 475/475 [00:02<00:00, 216.68it/s]
Epoch: 4. Train.      Loss: 0.925 , time : 24.076295614242554: 100%|██████████| 1898

Loss,█▆▅▄▃▃▃▂▂▂▂▁▁▁
Min_Val_Loss,██▁▁▁▁▁▁▁▁▁▁▁▁
Val_Loss,▆▆▁▂▂▂▆▃▁▂▅▃█▇
Loss,0.90222
Min_Val_Loss,3.95593
Val_Loss,3.9789


wandb: Agent Starting Run: hwevqa9m with config:
wandb: 	batch_size: 4
wandb: 	criterion: 0
wandb: 	learning_rate: 0.0001
wandb: 	model: 16
wandb: 	optimizer: 1
wandb: 	weight_decay: 1e-05


Epoch: 0. Train.      Loss: 0.902 , time : 24.397681713104248: 100%|██████████| 1898/1898 [00:24<00:00, 77.65it/s]
Epoch: 0. Validation. Loss: 3.980 , time : 2.1508724689483643: 100%|██████████| 475/475 [00:02<00:00, 216.43it/s]
Epoch: 1. Train.      Loss: 0.899 , time : 24.257630109786987: 100%|██████████| 1898/1898 [00:24<00:00, 78.10it/s]
Epoch: 1. Validation. Loss: 3.983 , time : 2.2045772075653076: 100%|██████████| 475/475 [00:02<00:00, 211.32it/s]
Epoch: 2. Train.      Loss: 0.905 , time : 21.31918215751648:  88%|████████▊ | 1673/1898 [00:21<00:02, 80.49it/s] 

In [13]:
hyper_params_setting

{'method': 'grid',
 'metric': {'name': 'Test_Loss', 'goal': 'minimize'},
 'parameters': {'batch_size': {'values': [4, 8, 16]},
  'learning_rate': {'values': [0.0001, 0.0003, 0.001, 0.003, 0.01, 0.03]},
  'weight_decay': {'values': [5e-06, 1e-05, 5e-05, 0.0001]},
  'criterion': {'values': [CrossEntropyLoss()]},
  'optimizer': {'values': [<function setting.exp_config.ExpConfig.__init__.<locals>.<lambda>(p, lr, wd)>,
    <function setting.exp_config.ExpConfig.__init__.<locals>.<lambda>(p, lr, wd)>]},
  'model': {'values': [KnowNet(
      (small_net_list): ModuleList(
        (0): Sequential(
          (0): Linear(in_features=2, out_features=4, bias=True)
          (1): ReLU()
          (2): Linear(in_features=4, out_features=1, bias=True)
          (3): ReLU()
        )
        (1): Sequential(
          (0): Linear(in_features=2, out_features=4, bias=True)
          (1): ReLU()
          (2): Linear(in_features=4, out_features=1, bias=True)
          (3): ReLU()
        )
        (2): Se